# GIFT × Riemann : Phase 3 - Validation & Résolution

**Objectifs** (recommandations Council-5) :
1. **Falsification** : Tester q=11 (non-GIFT) vs q=77 (GIFT)
2. **Ramanujan Δ** : Forme modulaire poids 12, niveau 1
3. **Toeplitz/Yule-Walker** : Prédicteur optimal sur séquence unfolded
4. **GUE Test** : Universalité vs structure arithmétique
5. **Dérivation analytique** : Pourquoi h_G₂² = 36 ?

**Data requise** :
- `zeros6` : 2M zéros de ζ(s) (Odlyzko)
- L-functions LMFDB : q=5, 7, 11, 21, 77, 248
- Ramanujan Δ zeros (LMFDB)

---

## Cellule 0 : Setup & Configuration

In [ ]:
# ============================================================
# SETUP - Exécuter en premier
# ============================================================

import numpy as np
import json
import os
from typing import List, Tuple, Dict, Optional
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

# Détection GPU (CuPy)
USE_GPU = False
try:
    import cupy as cp
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    USE_GPU = True
    xp = cp  # Use CuPy
    print("✓ GPU détecté (CuPy)")
except ImportError:
    xp = np  # Fallback to NumPy
    print("○ CPU mode (NumPy)")

# Constantes GIFT
@dataclass
class GIFTConstants:
    b3: int = 77          # Third Betti number of K₇
    dim_K7: int = 7       # Manifold dimension
    dim_G2: int = 14      # G₂ Lie algebra
    h_G2: int = 6         # Coxeter number
    rank_E8: int = 8      # E₈ rank
    dim_E8: int = 248     # E₈ dimension
    b2: int = 21          # Second Betti number
    dim_J3O: int = 27     # Jordan algebra
    H_star: int = 99      # b₂ + b₃ + 1
    
    # Derived
    h_G2_squared: int = 36  # 6²
    optimal_decimation: int = 24  # 3 × rank(E₈)
    
    # GIFT lags
    gift_lags: Tuple = (5, 8, 13, 27)
    standard_lags: Tuple = (1, 2, 3, 4)

GIFT = GIFTConstants()

print(f"\nConstantes GIFT chargées:")
print(f"   h_G₂² = {GIFT.h_G2_squared}")
print(f"   Lags GIFT = {GIFT.gift_lags}")
print(f"   Décimation optimale = {GIFT.optimal_decimation}")

## Cellule 1 : Fonctions utilitaires

In [ ]:
# ============================================================
# FONCTIONS UTILITAIRES
# ============================================================

def load_zeros(filepath: str) -> np.ndarray:
    """
    Charge les zéros depuis différents formats.
    Supporte: LMFDB JSON, texte simple, Odlyzko format.
    """
    zeros = []
    
    with open(filepath, 'r') as f:
        content = f.read().strip()
    
    # Chercher JSON
    lines = content.split('\n')
    json_start = 0
    for i, line in enumerate(lines):
        if line.strip().startswith('{') or line.strip().startswith('['):
            json_start = i
            break
    
    json_content = '\n'.join(lines[json_start:])
    
    if json_content.strip().startswith('{'):
        data = json.loads(json_content)
        if 'positive_zeros' in data:
            zeros = [float(z) for z in data['positive_zeros']]
    elif json_content.strip().startswith('['):
        zeros = [float(z) for z in json.loads(json_content)]
    else:
        # Format texte simple
        for line in lines:
            line = line.strip()
            if line and not line.startswith('#'):
                try:
                    zeros.append(float(line.split()[0]))
                except:
                    continue
    
    return np.array(sorted(zeros))


def fit_recurrence(gamma: np.ndarray, lags: List[int], 
                   start: int = None, end: int = None) -> Tuple[np.ndarray, float]:
    """
    Fit récurrence linéaire: γₙ = Σ aₖ·γₙ₋ₖ + c
    Retourne (coefficients, erreur moyenne).
    """
    max_lag = max(lags)
    if start is None:
        start = max_lag + 10
    if end is None:
        end = len(gamma)
    
    n_points = end - start
    n_params = len(lags) + 1
    
    X = np.zeros((n_points, n_params))
    for i, lag in enumerate(lags):
        X[:, i] = gamma[start - lag:end - lag]
    X[:, -1] = 1.0  # Constante
    
    y = gamma[start:end]
    coeffs, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    
    y_pred = X @ coeffs
    error = float(np.mean(np.abs(y_pred - y)))
    
    return coeffs, error


def compute_gift_metrics(coeffs: np.ndarray, lags: List[int]) -> Dict:
    """
    Calcule les métriques GIFT: produits lag×a, ratio 8/13, etc.
    """
    products = {lag: lag * coeffs[i] for i, lag in enumerate(lags)}
    
    metrics = {
        'coefficients': {f'a_{lag}': float(coeffs[i]) for i, lag in enumerate(lags)},
        'products': {f'{lag}×a_{lag}': float(products[lag]) for lag in lags},
        'c': float(coeffs[-1])
    }
    
    # Ratio 8×a₈ / 13×a₁₃
    if 8 in lags and 13 in lags:
        p8, p13 = products[8], products[13]
        if p13 != 0:
            ratio = p8 / p13
            metrics['ratio_8_13'] = float(ratio)
            metrics['deviation_from_1'] = float(abs(ratio - 1))
    
    # Somme des produits
    metrics['sum_products'] = float(sum(products.values()))
    
    return metrics


def N_riemann(T: float) -> float:
    """
    Fonction de comptage de Riemann-von Mangoldt N(T).
    Nombre de zéros avec 0 < γ < T.
    """
    if T <= 0:
        return 0
    return (T / (2 * np.pi)) * (np.log(T / (2 * np.pi)) - 1) + 7/8


def unfold_zeros(gamma: np.ndarray) -> np.ndarray:
    """
    Dépliement: γₙ → uₙ = N(γₙ)
    Transforme en séquence à densité uniforme.
    """
    return np.array([N_riemann(g) for g in gamma])


print("✓ Fonctions utilitaires chargées")

## Cellule 2 : Upload des données (Colab)

In [ ]:
# ============================================================
# UPLOAD DES DONNÉES (Colab)
# ============================================================

# Créer le répertoire data/
os.makedirs('data', exist_ok=True)

# Sur Colab: décommenter pour upload
# from google.colab import files
# uploaded = files.upload()  # Upload tes fichiers ici
# for name in uploaded:
#     os.rename(name, f'data/{name}')

# Liste des fichiers attendus
EXPECTED_FILES = {
    'zeros': [
        'data/zeros1',           # 100k zéros ζ(s)
        'data/zeros6',           # 2M zéros ζ(s)
    ],
    'dirichlet': [
        'data/1-5-5.4-r0-0-0.zeros.txt',
        'data/1-7-7.6-r1-0-0.zeros.txt',
        'data/1-11-*.zeros.txt',      # À télécharger
        'data/1-21-21.20-r0-0-0.zeros.txt',
        'data/1-77-77.76-r0-0-0.zeros.txt',
        'data/1-99-*.zeros.txt',      # À télécharger si dispo
        'data/1-248-248.123-r0-0-0.zeros.txt',
    ],
    'modular': [
        'data/ramanujan_delta.zeros.txt',  # Forme modulaire Δ
    ]
}

# Vérifier ce qui est disponible
print("Fichiers disponibles:")
import glob
for f in glob.glob('data/*'):
    size = os.path.getsize(f)
    print(f"   {f}: {size:,} bytes")

# Charger automatiquement ce qui est disponible
DATA = {}

# Zéros de zeta
for name in ['zeros1', 'zeros6']:
    path = f'data/{name}'
    if os.path.exists(path):
        DATA[name] = load_zeros(path)
        print(f"\n✓ {name}: {len(DATA[name]):,} zéros chargés")

# L-functions Dirichlet
for f in glob.glob('data/1-*-*.zeros.txt') + glob.glob('data/1-*-*.zeros.json'):
    try:
        # Extraire conducteur du nom
        parts = os.path.basename(f).split('-')
        q = int(parts[1])
        zeros = load_zeros(f)
        DATA[f'L_q{q}'] = zeros
        print(f"✓ L(s,χ_{q}): {len(zeros)} zéros")
    except:
        continue

print(f"\n=== {len(DATA)} datasets chargés ===")

---
# PARTIE 1 : Tests de Falsification
---

## Cellule 3 : Test q=11 vs q=77 (Falsification)

In [ ]:
# ============================================================
# TEST DE FALSIFICATION : q=11 (non-GIFT) vs q=77 (GIFT)
# ============================================================
# 
# Hypothèse GIFT: Seuls les conducteurs liés aux constantes GIFT
# (5, 7, 14, 21, 27, 77, 248) devraient bien satisfaire la contrainte.
#
# Si q=11 (premier, non-GIFT) donne un meilleur ratio que q=77,
# alors GIFT est falsifié.
# ============================================================

def analyze_conductor(zeros: np.ndarray, q: int, verbose: bool = True) -> Dict:
    """
    Analyse complète d'une L-function.
    """
    result = {
        'conductor': q,
        'n_zeros': len(zeros),
        'is_gift_related': q in [5, 7, 14, 21, 27, 77, 248, 99]
    }
    
    if len(zeros) < 50:
        result['status'] = 'insufficient_data'
        return result
    
    # GIFT analysis
    gift_lags = list(GIFT.gift_lags)
    std_lags = list(GIFT.standard_lags)
    
    coeffs_gift, err_gift = fit_recurrence(zeros, gift_lags)
    coeffs_std, err_std = fit_recurrence(zeros, std_lags)
    
    gift_metrics = compute_gift_metrics(coeffs_gift, gift_lags)
    
    result['gift'] = {
        'error': err_gift,
        'metrics': gift_metrics
    }
    result['standard'] = {'error': err_std}
    result['gift_wins'] = err_gift < err_std
    result['improvement_pct'] = (err_std - err_gift) / err_std * 100 if err_std > 0 else 0
    
    if verbose:
        print(f"\n{'='*60}")
        print(f"CONDUCTEUR q = {q} {'(GIFT)' if result['is_gift_related'] else '(non-GIFT)'}")
        print(f"{'='*60}")
        print(f"   N zéros: {len(zeros)}")
        print(f"   GIFT gagne: {'OUI' if result['gift_wins'] else 'NON'} ({result['improvement_pct']:.1f}%)")
        if 'ratio_8_13' in gift_metrics:
            print(f"   Ratio 8×a₈/13×a₁₃ = {gift_metrics['ratio_8_13']:.4f}")
            print(f"   Déviation |R-1| = {gift_metrics['deviation_from_1']*100:.1f}%")
    
    return result

# Analyser tous les conducteurs disponibles
results_falsification = {}

for key, zeros in DATA.items():
    if key.startswith('L_q'):
        q = int(key.split('q')[1])
        results_falsification[q] = analyze_conductor(zeros, q)

# Résumé comparatif
print("\n" + "="*70)
print("RÉSUMÉ : FALSIFICATION TEST")
print("="*70)
print(f"\n{'q':<8} {'GIFT?':<8} {'N':<8} {'|R-1|':<12} {'GIFT wins':<12}")
print("-"*50)

for q in sorted(results_falsification.keys()):
    r = results_falsification[q]
    is_gift = '✓' if r['is_gift_related'] else '✗'
    n = r['n_zeros']
    dev = r['gift']['metrics'].get('deviation_from_1', None)
    dev_str = f"{dev*100:.1f}%" if dev else "N/A"
    wins = '✓' if r['gift_wins'] else '✗'
    print(f"{q:<8} {is_gift:<8} {n:<8} {dev_str:<12} {wins:<12}")

# Verdict
print("\n" + "="*70)
print("VERDICT")
print("="*70)

gift_conductors = [q for q, r in results_falsification.items() if r['is_gift_related']]
non_gift_conductors = [q for q, r in results_falsification.items() if not r['is_gift_related']]

if gift_conductors and non_gift_conductors:
    avg_gift = np.mean([results_falsification[q]['gift']['metrics'].get('deviation_from_1', 1) 
                        for q in gift_conductors])
    avg_non_gift = np.mean([results_falsification[q]['gift']['metrics'].get('deviation_from_1', 1) 
                           for q in non_gift_conductors])
    
    print(f"\n   Déviation moyenne (GIFT): {avg_gift*100:.1f}%")
    print(f"   Déviation moyenne (non-GIFT): {avg_non_gift*100:.1f}%")
    
    if avg_gift < avg_non_gift:
        print(f"\n   → GIFT CONFIRMÉ: conducteurs GIFT meilleurs ({avg_gift*100:.1f}% < {avg_non_gift*100:.1f}%)")
    else:
        print(f"\n   → GIFT NON CONFIRMÉ: pas de différence significative")
else:
    print("\n   Besoin de plus de conducteurs pour conclure")
    print("   Télécharger q=11 depuis LMFDB: https://www.lmfdb.org/L/1/11/")

## Cellule 4 : Classement complet des conducteurs

In [ ]:
# ============================================================
# CLASSEMENT COMPLET DES CONDUCTEURS
# ============================================================

print("="*70)
print("CLASSEMENT PAR DÉVIATION FIBONACCI |R-1|")
print("="*70)

# Créer liste triée
ranking = []
for q, r in results_falsification.items():
    dev = r['gift']['metrics'].get('deviation_from_1', None)
    if dev is not None:
        ranking.append({
            'q': q,
            'deviation': dev,
            'is_gift': r['is_gift_related'],
            'n_zeros': r['n_zeros'],
            'gift_wins': r['gift_wins'],
            'improvement': r['improvement_pct']
        })

ranking.sort(key=lambda x: x['deviation'])

print(f"\n{'Rank':<6} {'q':<8} {'|R-1|':<10} {'GIFT?':<8} {'N':<8} {'Amélioration'}")
print("-"*60)

for i, r in enumerate(ranking):
    gift_marker = '★' if r['is_gift'] else ' '
    print(f"{i+1:<6} q={r['q']:<5} {r['deviation']*100:>6.1f}%   {gift_marker:<8} {r['n_zeros']:<8} +{r['improvement']:.1f}%")

# Interprétation GIFT
print("\n" + "="*70)
print("INTERPRÉTATION GIFT")
print("="*70)

gift_meanings = {
    5: 'Premier lag GIFT (F₅)',
    7: 'dim(K₇)',
    14: 'dim(G₂)',
    21: 'b₂',
    27: 'dim(J₃(O))',
    77: 'b₃ ★',
    99: 'H* = b₂+b₃+1',
    248: 'dim(E₈)'
}

for r in ranking:
    q = r['q']
    if q in gift_meanings:
        print(f"   q={q}: {gift_meanings[q]} → dév. {r['deviation']*100:.1f}%")

---
# PARTIE 2 : Analyse Toeplitz/Yule-Walker
---

## Cellule 5 : Séquence Unfolded et Autocovariance

In [ ]:
# ============================================================
# ANALYSE TOEPLITZ/YULE-WALKER SUR SÉQUENCE UNFOLDED
# ============================================================
#
# Recommandation GPT: Travailler sur u_n = N(γ_n), pas γ_n
# Le prédicteur optimal devrait révéler la structure.
# ============================================================

def compute_autocovariance(x: np.ndarray, max_lag: int) -> np.ndarray:
    """
    Calcule l'autocovariance C(k) = Cov(x_n, x_{n-k})
    """
    n = len(x)
    x_centered = x - np.mean(x)
    
    C = np.zeros(max_lag + 1)
    for k in range(max_lag + 1):
        C[k] = np.mean(x_centered[k:] * x_centered[:n-k]) if k < n else 0
    
    return C


def solve_yule_walker(C: np.ndarray, lags: List[int]) -> np.ndarray:
    """
    Résout le système Yule-Walker pour obtenir les coefficients optimaux.
    
    T·a = c  où T_ij = C(|lag_i - lag_j|), c_i = C(lag_i)
    """
    n_lags = len(lags)
    
    # Matrice Toeplitz
    T = np.zeros((n_lags, n_lags))
    for i, li in enumerate(lags):
        for j, lj in enumerate(lags):
            T[i, j] = C[abs(li - lj)]
    
    # Vecteur c
    c = np.array([C[lag] for lag in lags])
    
    # Résoudre
    try:
        a = np.linalg.solve(T, c)
    except np.linalg.LinAlgError:
        a = np.linalg.lstsq(T, c, rcond=None)[0]
    
    return a


# Charger les données principales
if 'zeros6' in DATA:
    gamma = DATA['zeros6']
    source = 'zeros6 (2M)'
elif 'zeros1' in DATA:
    gamma = DATA['zeros1']
    source = 'zeros1 (100k)'
else:
    print("Aucun fichier de zéros ζ(s) trouvé!")
    gamma = None

if gamma is not None:
    print(f"Source: {source}")
    print(f"N = {len(gamma):,} zéros")
    
    # Unfold
    print("\nUnfolding γ → u = N(γ)...")
    u = unfold_zeros(gamma)
    
    # Déviation x_n = u_n - n
    x = u - np.arange(len(u))
    
    print(f"   mean(x) = {np.mean(x):.4f}")
    print(f"   std(x) = {np.std(x):.4f}")
    
    # Autocovariance
    print("\nCalcul autocovariance...")
    max_lag = 50
    C = compute_autocovariance(x, max_lag)
    
    print(f"   C(0) = {C[0]:.6f} (variance)")
    print(f"   C(1) = {C[1]:.6f}")
    print(f"   C(5) = {C[5]:.6f}")
    print(f"   C(8) = {C[8]:.6f}")
    print(f"   C(13) = {C[13]:.6f}")
    print(f"   C(27) = {C[27]:.6f}")

## Cellule 6 : Yule-Walker - Coefficients optimaux

In [ ]:
# ============================================================
# YULE-WALKER : COEFFICIENTS OPTIMAUX
# ============================================================

if gamma is not None and 'C' in dir():
    print("="*70)
    print("YULE-WALKER : PRÉDICTEUR OPTIMAL")
    print("="*70)
    
    # GIFT lags
    gift_lags = [5, 8, 13, 27]
    a_gift_yw = solve_yule_walker(C, gift_lags)
    
    print(f"\nLags GIFT {gift_lags}:")
    for i, lag in enumerate(gift_lags):
        print(f"   a_{lag} (Yule-Walker) = {a_gift_yw[i]:.6f}")
    
    # Produits
    products_yw = {lag: lag * a_gift_yw[i] for i, lag in enumerate(gift_lags)}
    print(f"\nProduits lag × a:")
    for lag in gift_lags:
        print(f"   {lag} × a_{lag} = {products_yw[lag]:.4f}")
    
    # Ratio 8/13
    ratio_yw = products_yw[8] / products_yw[13] if products_yw[13] != 0 else float('inf')
    print(f"\n   Ratio (8×a₈)/(13×a₁₃) = {ratio_yw:.4f}")
    print(f"   Déviation |R-1| = {abs(ratio_yw - 1)*100:.2f}%")
    
    # Comparaison avec régression directe
    print("\n" + "-"*50)
    print("COMPARAISON: Yule-Walker vs Régression directe")
    print("-"*50)
    
    # Régression sur x (unfolded)
    coeffs_reg, err_reg = fit_recurrence(x, gift_lags)
    products_reg = {lag: lag * coeffs_reg[i] for i, lag in enumerate(gift_lags)}
    ratio_reg = products_reg[8] / products_reg[13] if products_reg[13] != 0 else float('inf')
    
    print(f"\n{'Méthode':<20} {'Ratio 8/13':<12} {'|R-1|':<10}")
    print("-"*45)
    print(f"{'Yule-Walker':<20} {ratio_yw:<12.4f} {abs(ratio_yw-1)*100:<10.2f}%")
    print(f"{'Régression':<20} {ratio_reg:<12.4f} {abs(ratio_reg-1)*100:<10.2f}%")
    
    # Standard lags pour comparaison
    std_lags = [1, 2, 3, 4]
    a_std_yw = solve_yule_walker(C, std_lags)
    
    print(f"\n{'='*70}")
    print(f"COMPARAISON LAGS: GIFT vs Standard (Yule-Walker)")
    print(f"{'='*70}")
    
    # Erreur de prédiction
    def prediction_error(x, lags, coeffs):
        max_lag = max(lags)
        n = len(x)
        errors = []
        for i in range(max_lag, min(n, max_lag + 10000)):
            pred = sum(coeffs[j] * x[i - lags[j]] for j in range(len(lags)))
            errors.append(abs(x[i] - pred))
        return np.mean(errors)
    
    err_gift_yw = prediction_error(x, gift_lags, a_gift_yw)
    err_std_yw = prediction_error(x, std_lags, a_std_yw)
    
    print(f"\n   Erreur GIFT lags: {err_gift_yw:.6f}")
    print(f"   Erreur Standard lags: {err_std_yw:.6f}")
    print(f"   GIFT {'gagne' if err_gift_yw < err_std_yw else 'perd'}: {(err_std_yw - err_gift_yw)/err_std_yw*100:.1f}%")

---
# PARTIE 3 : Test GUE (Universalité)
---

## Cellule 7 : Génération matrices GUE et comparaison

In [ ]:
# ============================================================
# TEST GUE : UNIVERSALITÉ
# ============================================================
#
# Question: La structure GIFT est-elle spécifique à ζ(s) ou
# universelle (présente dans toute matrice GUE) ?
#
# Si GUE montre la même structure → Universel (classe GUE)
# Si GUE différent → Arithmétique (spécifique à ζ)
# ============================================================

def generate_gue_eigenvalues(n: int, n_matrices: int = 10) -> np.ndarray:
    """
    Génère les valeurs propres de matrices GUE.
    GUE = Gaussian Unitary Ensemble.
    """
    all_eigenvalues = []
    
    for _ in range(n_matrices):
        # Matrice hermitienne aléatoire
        A = np.random.randn(n, n) + 1j * np.random.randn(n, n)
        H = (A + A.conj().T) / 2
        
        # Valeurs propres (réelles pour matrice hermitienne)
        eigenvalues = np.linalg.eigvalsh(H)
        all_eigenvalues.extend(eigenvalues)
    
    return np.array(sorted(all_eigenvalues))


def unfold_gue(eigenvalues: np.ndarray) -> np.ndarray:
    """
    Unfold GUE eigenvalues using semicircle law.
    N(E) = (n/π) * (E*sqrt(1-E²/4) + 2*arcsin(E/2)) for |E| < 2√n
    Simplified: use empirical CDF.
    """
    n = len(eigenvalues)
    # Empirical unfolding: rank / n
    return np.arange(n).astype(float)


print("="*70)
print("TEST GUE : UNIVERSALITÉ")
print("="*70)

# Générer eigenvalues GUE
print("\nGénération matrices GUE...")
n_matrix = 500  # Taille de chaque matrice
n_matrices = 20  # Nombre de matrices

gue_eigenvalues = generate_gue_eigenvalues(n_matrix, n_matrices)
print(f"   {len(gue_eigenvalues):,} valeurs propres GUE générées")

# Unfold
gue_unfolded = unfold_gue(gue_eigenvalues)
gue_x = gue_eigenvalues  # Utiliser directement les eigenvalues triées

# Autocovariance GUE
print("\nCalcul autocovariance GUE...")
C_gue = compute_autocovariance(gue_x, 50)

# Yule-Walker sur GUE
gift_lags = [5, 8, 13, 27]
a_gue = solve_yule_walker(C_gue, gift_lags)

print(f"\nCoefficients Yule-Walker (GUE):")
for i, lag in enumerate(gift_lags):
    print(f"   a_{lag} = {a_gue[i]:.6f}")

# Produits et ratio
products_gue = {lag: lag * a_gue[i] for i, lag in enumerate(gift_lags)}
ratio_gue = products_gue[8] / products_gue[13] if products_gue[13] != 0 else float('inf')

print(f"\nProduits lag × a (GUE):")
for lag in gift_lags:
    print(f"   {lag} × a_{lag} = {products_gue[lag]:.4f}")

print(f"\n   Ratio (8×a₈)/(13×a₁₃) = {ratio_gue:.4f}")
print(f"   Déviation |R-1| = {abs(ratio_gue - 1)*100:.2f}%")

# Comparaison ζ vs GUE
print("\n" + "="*70)
print("COMPARAISON: ζ(s) vs GUE")
print("="*70)

if 'ratio_yw' in dir():
    print(f"\n{'Source':<15} {'Ratio 8/13':<12} {'|R-1|':<10}")
    print("-"*40)
    print(f"{'ζ(s)':<15} {ratio_yw:<12.4f} {abs(ratio_yw-1)*100:<10.2f}%")
    print(f"{'GUE':<15} {ratio_gue:<12.4f} {abs(ratio_gue-1)*100:<10.2f}%")
    
    # Verdict
    diff = abs(ratio_yw - ratio_gue)
    if diff < 0.1:
        print(f"\n   → UNIVERSEL: ζ(s) et GUE similaires (diff = {diff:.3f})")
    else:
        print(f"\n   → ARITHMÉTIQUE: ζ(s) différent de GUE (diff = {diff:.3f})")

---
# PARTIE 4 : Analyse G₂ et origine de h_G₂² = 36
---

## Cellule 8 : Invariants G₂ et dérivation analytique

In [ ]:
# ============================================================
# INVARIANTS G₂ ET DÉRIVATION ANALYTIQUE
# ============================================================
#
# Question: Pourquoi 8×β₈ = 13×β₁₃ = 36 = h_G₂² ?
#
# Hypothèse (Grok): β_i = h_G₂² / lag_i
# Vérifions si c'est cohérent.
# ============================================================

print("="*70)
print("ANALYSE G₂ : ORIGINE DE h_G₂² = 36")
print("="*70)

# Constantes G₂
h_G2 = 6        # Coxeter number
dim_G2 = 14     # Dimension
rank_G2 = 2     # Rank

print(f"\nConstantes G₂:")
print(f"   h_G₂ = {h_G2} (Coxeter number)")
print(f"   dim(G₂) = {dim_G2}")
print(f"   rank(G₂) = {rank_G2}")
print(f"   h_G₂² = {h_G2**2}")

# Identités remarquables
print(f"\nIdentités:")
print(f"   h_G₂ × (h_G₂ + 1) = {h_G2 * (h_G2 + 1)} = 3 × dim(G₂) = {3 * dim_G2}")
print(f"   dim(G₂) = 2 × dim(K₇) = {2 * 7}")
print(f"   h_G₂² / dim(G₂) = {h_G2**2 / dim_G2:.4f}")

# Matrice de Cartan G₂
print(f"\nMatrice de Cartan G₂:")
cartan_G2 = np.array([[2, -3], 
                       [-1, 2]])
print(cartan_G2)
print(f"   det = {np.linalg.det(cartan_G2):.0f}")
print(f"   trace = {np.trace(cartan_G2):.0f}")

# Racines de G₂
print(f"\nRacines simples G₂ (norme):")
alpha1_norm = 2  # Racine courte
alpha2_norm = 6  # Racine longue
print(f"   |α₁|² = {alpha1_norm} (courte)")
print(f"   |α₂|² = {alpha2_norm} (longue)")
print(f"   |α₂|² = h_G₂ × 1 = {h_G2}")

# Test: β_i = h_G₂² / lag_i
print(f"\n{'='*70}")
print("TEST: β_i = h_G₂² / lag_i")
print("="*70)

# Valeurs empiriques de Phase 2.6
beta_empirical = {
    5: 0.767,
    8: 4.497,
    13: 2.764,
    27: 3.106
}

print(f"\n{'Lag':<6} {'β empirique':<12} {'h_G₂²/lag':<12} {'Erreur':<10}")
print("-"*45)

h_G2_sq = 36
for lag, beta_emp in beta_empirical.items():
    beta_pred = h_G2_sq / lag
    error = abs(beta_emp - beta_pred) / beta_emp * 100
    print(f"{lag:<6} {beta_emp:<12.4f} {beta_pred:<12.4f} {error:<10.1f}%")

# Vérification 8×β₈ = 13×β₁₃
print(f"\nVérification contrainte Fibonacci:")
print(f"   8 × β₈ = 8 × {beta_empirical[8]:.3f} = {8 * beta_empirical[8]:.2f}")
print(f"   13 × β₁₃ = 13 × {beta_empirical[13]:.3f} = {13 * beta_empirical[13]:.2f}")
print(f"   Ratio = {8 * beta_empirical[8] / (13 * beta_empirical[13]):.4f}")

# Si β_i = h_G₂² / lag_i, alors lag × β = h_G₂² pour tous
print(f"\nSi β_i = h_G₂² / lag_i:")
print(f"   → lag × β = h_G₂² = 36 pour TOUS les lags")
print(f"   → Contrainte 8×β₈ = 13×β₁₃ automatiquement satisfaite!")

## Cellule 9 : Combinatoire 24 et 36

In [ ]:
# ============================================================
# COMBINATOIRE: 24 ET 36
# ============================================================
#
# Observations (Opus):
# - 24 + 36 = 60 = |A₅| (groupe alterné/icosaèdre)
# - 24 = kissing number dim 4 = 3 × rank(E₈)
# - 36 = h_G₂²
# ============================================================

print("="*70)
print("COMBINATOIRE: 24 ET 36")
print("="*70)

# Identités
print(f"\n24:")
print(f"   24 = 3 × 8 = 3 × rank(E₈)")
print(f"   24 = 4 × 6 = 4 × h_G₂")
print(f"   24 = kissing number en dimension 4")
print(f"   24 = |SL(2,Z/3)| / ... ")
print(f"   24 = poids de Ramanujan Δ / 0.5")

print(f"\n36:")
print(f"   36 = 6² = h_G₂²")
print(f"   36 = 4 × 9 = 4 × 3²")
print(f"   36 = dim(so(9)) - dim(so(8)) + 1 = {36 - 28 + 1}")

print(f"\n24 + 36 = {24 + 36}:")
print(f"   60 = |A₅| (groupe alterné sur 5 éléments)")
print(f"   60 = ordre du groupe d'icosaèdre")
print(f"   60 = 5 × 12 = 5 × (h_G₂ × 2)")

print(f"\n24 × 36 = {24 * 36}:")
print(f"   864 = 2⁵ × 3³ = 32 × 27")
print(f"   864 = dim(E₈) × 3.48...")

print(f"\n36 - 24 = {36 - 24}:")
print(f"   12 = 2 × h_G₂")
print(f"   12 = poids de Ramanujan Δ")
print(f"   12 = |faces du dodécaèdre|")

print(f"\n36 / 24 = {36 / 24}:")
print(f"   1.5 = 3/2")

# Lien avec lags GIFT
print(f"\n{'='*70}")
print("LIEN AVEC LAGS GIFT")
print("="*70)

print(f"\nLags: [5, 8, 13, 27]")
print(f"   Somme = {5 + 8 + 13 + 27} = 53 (premier)")
print(f"   Produit = {5 * 8 * 13 * 27} = 14040 = 8 × 1755")
print(f"   8 × 13 = {8 * 13} = 104 = 4 × 26")
print(f"   5 × 27 = {5 * 27} = 135 = 5 × 27")
print(f"   8 + 13 = {8 + 13} = b₂")
print(f"   5 + 8 = {5 + 8} = F₇")
print(f"   8 + 13 + 27 = {8 + 13 + 27} = 48 = 2 × 24")

---
# PARTIE 5 : Export des résultats
---

## Cellule 10 : Synthèse et export JSON

In [ ]:
# ============================================================
# SYNTHÈSE ET EXPORT
# ============================================================

print("="*70)
print("SYNTHÈSE PHASE 3")
print("="*70)

results_phase3 = {
    'meta': {
        'date': '2026-01',
        'phase': '3',
        'status': 'in_progress'
    },
    'falsification': {},
    'toeplitz': {},
    'gue_test': {},
    'g2_analysis': {}
}

# Falsification
if 'results_falsification' in dir():
    results_phase3['falsification'] = {
        'conductors_tested': list(results_falsification.keys()),
        'ranking': ranking if 'ranking' in dir() else []
    }

# Toeplitz
if 'ratio_yw' in dir():
    results_phase3['toeplitz'] = {
        'ratio_yule_walker': float(ratio_yw),
        'deviation': float(abs(ratio_yw - 1)),
        'coefficients': {f'a_{lag}': float(a_gift_yw[i]) for i, lag in enumerate(gift_lags)}
    }

# GUE
if 'ratio_gue' in dir():
    results_phase3['gue_test'] = {
        'ratio_gue': float(ratio_gue),
        'ratio_zeta': float(ratio_yw) if 'ratio_yw' in dir() else None,
        'difference': float(abs(ratio_yw - ratio_gue)) if 'ratio_yw' in dir() else None,
        'verdict': 'universal' if 'ratio_yw' in dir() and abs(ratio_yw - ratio_gue) < 0.1 else 'arithmetic'
    }

# G₂
results_phase3['g2_analysis'] = {
    'h_G2_squared': 36,
    'beta_prediction': 'β_i = h_G₂² / lag_i',
    'identities': {
        '24_plus_36': 60,
        '24_times_36': 864,
        '36_minus_24': 12
    }
}

# Sauvegarder
with open('phase3_results.json', 'w') as f:
    json.dump(results_phase3, f, indent=2, default=str)

print("\n✓ Résultats sauvegardés dans phase3_results.json")

# Afficher résumé
print(f"\n{'='*70}")
print("RÉSUMÉ EXÉCUTIF")
print("="*70)

print(f"\n1. FALSIFICATION:")
if 'ranking' in dir() and ranking:
    best = ranking[0]
    print(f"   Meilleur conducteur: q={best['q']} (dév. {best['deviation']*100:.1f}%)")

print(f"\n2. TOEPLITZ/YULE-WALKER:")
if 'ratio_yw' in dir():
    print(f"   Ratio 8/13 = {ratio_yw:.4f} (dév. {abs(ratio_yw-1)*100:.2f}%)")

print(f"\n3. GUE TEST:")
if 'ratio_gue' in dir():
    verdict = results_phase3['gue_test']['verdict']
    print(f"   Verdict: {verdict.upper()}")

print(f"\n4. G₂ ANALYSIS:")
print(f"   Hypothèse β_i = 36/lag_i: À VÉRIFIER avec données RG flow")